# 让类型对象支持排序

Python 3中类型不能排序，下面的代码抛出`TypeError`异常：

In [1]:
class A:
    pass

class B:
    pass

class C:
    pass

class Z:
    pass

classes = [Z, B, A, C]
sorted(classes)

TypeError: unorderable types: type() < type()

我们知道对象的比较行为由其类型中的魔法方法`__lt__（）`决定。而在`type`对象的字典中没有`__lt__`：

In [2]:
"__lt__" in type.__dict__

False

下面我们定义`__lt__()`函数并将其添加进`type.__dict__`，但是抛出异常。这是由于Python对类型对象的字典进行了写保护：

In [3]:
def __lt__(self, other):
    return self.__name__ < other.__name__

type.__dict__["__lt__"] = __lt__

TypeError: 'mappingproxy' object does not support item assignment

`type.__dict__`是一个`mappingproxy`对象，它对字典对象进行只读包装：

In [4]:
type(type.__dict__)

mappingproxy

为了往`type`的字典中添加内容，必须获得该字典的引用。可以使用`gc.get_referents()`实现。`get_referents(obj)`可以获得`obj`对象引用的所有其它对象。因此可以使用它获取`mappingproxy`对象内部包装的字典对象。下面将`__lt__()`函数添加进行`type`的字典中：

In [5]:
import gc
type_dict = gc.get_referents(type.__dict__)[0]
type_dict["__lt__"] = __lt__
"__lt__" in type.__dict__

True

然而即使在字典中添加了`__lt__`函数，仍然无法对类型对象进行排序。

In [6]:
sorted(classes)

TypeError: unorderable types: type() < type()

对象要支持比较运算符，必须满足以下两个条件：

1. 对象的类的`__dict__`中包含名为`__lt__`的函数。
2. 类对应的结构体的字段`tp_richcompare`保存[`slot_tp_richcompare()`](https://github.com/python/cpython/blob/4e624ca50a665d7e4d527ab98932347ff43a19b0/Objects/typeobject.c#L6378)函数的地址，该函数在`typeobject.c`文件中定义。

这里我们希望让类型对象支持比较符，因此需要在类型对象的类型`type`对应的结构体`PyTypeObject`的`tp_richcompare`字段中保存`slot_tp_richcompare()`。下面看看如何修改`tp_richcompare`字段。首先需要获取该字段的偏移地址：

In [7]:
import cffi

ffi = cffi.FFI()
ffi.cdef("""
ssize_t tp_richcompare;
""")

lib = ffi.verify("""
ssize_t tp_richcompare = offsetof(PyTypeObject, tp_richcompare);
""")

lib.tp_richcompare

200

由于`slot_tp_richcompare()`为`static`函数，无法直接获取其地址。因此这里定义一个`Dummy`类，其`tp_richcompare`字段中保存的就是`slot_tp_richcompare()`的地址：

In [9]:
class Dummy:
    def __lt__(self, other):
        return id(self) < id(other)
    
class Dummy2(Dummy):
    pass

class Dummy3:
    def __lt__(self, other):
        return id(self) < id(other)

下面获取函数地址，并与`Dummy2`和`Dummy3`的字段中的值比较，它们都是同一个地址：

In [10]:
addr_slot_tp_richcompare = int(ffi.cast("ssize_t *", id(Dummy) + lib.tp_richcompare)[0])

assert addr_slot_tp_richcompare == int(ffi.cast("ssize_t *", id(Dummy2) + lib.tp_richcompare)[0])
assert addr_slot_tp_richcompare == int(ffi.cast("ssize_t *", id(Dummy3) + lib.tp_richcompare)[0])

下面将`addr_slot_tp_richcompare`写入到`type`对应的结构体的`tp_richcompare`字段中：

In [11]:
ffi.cast("ssize_t *", id(type) + lib.tp_richcompare)[0] = addr_slot_tp_richcompare

于是就可以对类进行排序了：

In [12]:
sorted(classes)

[__main__.A, __main__.B, __main__.C, __main__.Z]